In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
from senmodel.model.utils import convert_dense_to_sparse_network, get_model_last_layer
from senmodel.metrics.nonlinearity_metrics import GradientMeanEdgeMetric, PerturbationSensitivityEdgeMetric
from senmodel.metrics.edge_finder import EdgeFinder


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

scaler = StandardScaler()


y = data['occupation']
y = LabelEncoder().fit_transform(y)

X = data.drop(['occupation'], axis=1)

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

len(set(y))

15

In [5]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [7]:
class MulticlassFCN(nn.Module):
    def __init__(self, input_size=14, hidden_sizes=None, output_size=15, dropout_rate=0.3):
        super(MulticlassFCN, self).__init__()
        if hidden_sizes is None:
            hidden_sizes = [128, 64]
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.1)

        self.output = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.output(x)
        # x = self.dropout2(x)
        return x


In [8]:
def edge_replacement_func_new_layer(model, optim, val_loader, metric):
    layer = get_model_last_layer(model)
    ef = EdgeFinder(metric, val_loader, device)
    vals = ef.calculate_edge_metric_for_dataloader(model)
    print("Edge metrics:", vals, max(vals), sum(vals))
    chosen_edges = ef.choose_edges_threshold(model, 0.2)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if layer.embed_linears:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
    else:
        print("Empty metric")
        dummy_param = torch.zeros_like(layer.weight_values)
        optim.add_param_group({'params': dummy_param})

    return {'max': max(vals), 'sum': sum(vals), 'len': len(vals), 'len_choose': len(chosen_edges[0])}


In [9]:
from senmodel.model.utils import freeze_all_but_last, unfreeze_all


def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None,
                           window_size=3, threshold=0.1, fine_tune_epochs=3):
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    criterion = nn.CrossEntropyLoss()
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets in tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        val_accuracy = accuracy_score(all_targets, all_preds)
        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

        new_l = {}
        val_losses.append(val_loss)
        if edge_replacement_func and len(val_losses) > window_size:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                new_l = edge_replacement_func(model, optimizer, val_loader, metric)
                # Замораживаем все слои кроме последнего
                freeze_all_but_last(model)

                # Обучаем только последний слой в течение нескольких эпох
                for fine_tune_epoch in range(fine_tune_epochs):
                    model.train()
                    fine_tune_train_loss = 0
                    for inputs, targets in tqdm(train_loader):
                        inputs, targets = inputs.to(device), targets.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, targets)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        fine_tune_train_loss += loss.item()
                    fine_tune_train_loss /= len(train_loader)

                    model.eval()
                    fine_tune_val_loss = 0
                    with torch.no_grad():
                        for inputs, targets in val_loader:
                            inputs, targets = inputs.to(device), targets.to(device)
                            outputs = model(inputs)
                            loss = criterion(outputs, targets)
                            fine_tune_val_loss += loss.item()
                    fine_tune_val_loss /= len(val_loader)

                    print(f"Fine-Tune Epoch {fine_tune_epoch + 1}/{fine_tune_epochs} | "
                          f"Train Loss: {fine_tune_train_loss:.4f} | Val Loss: {fine_tune_val_loss:.4f}")

                # Размораживаем все слои
                unfreeze_all(model)
        #

        # new_l = {}
        # if edge_replacement_func and epoch % 8 == 0 and epoch != 0:
        #     new_l = edge_replacement_func(model, optimizer, val_loader, metric)

        wandb.log({'val_loss': val_loss, 'val_accuracy': val_accuracy, 'train_loss': train_loss} | new_l)



In [10]:
criterion = nn.CrossEntropyLoss()
metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]


In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk. Use `wandb login --relogin` to force relogin


True

In [12]:
dense_model = MulticlassFCN(input_size=X.shape[1])
sparse_model = convert_dense_to_sparse_network(dense_model)
wandb.init(
    project="self-expanding-nets",
    name=f"titanic, multiclass_complex_model, no dropout no change",
)

train_sparse_recursive(sparse_model, train_loader, val_loader, num_epochs=100,
                       metric=metrics[0],
                       edge_replacement_func=edge_replacement_func_new_layer)

100%|██████████| 51/51 [00:02<00:00, 22.54it/s]


Epoch 1/100 | Train Loss: 2.6047 | Val Loss: 31.5237 | Val Accuracy: 0.2016


100%|██████████| 51/51 [00:02<00:00, 22.26it/s]


Epoch 2/100 | Train Loss: 2.3399 | Val Loss: 28.2204 | Val Accuracy: 0.3232


100%|██████████| 51/51 [00:02<00:00, 22.76it/s]


...

100%|██████████| 51/51 [00:06<00:00,  7.56it/s]


Fine-Tune Epoch 3/3 | Train Loss: 1.9824 | Val Loss: 1.8960


100%|██████████| 51/51 [00:08<00:00,  6.07it/s]


Epoch 60/100 | Train Loss: 1.9628 | Val Loss: 24.4403 | Val Accuracy: 0.3627
Edge metrics: tensor([0.0000e+00, 3.2856e-02, 1.1170e-04, 3.3524e-02, 3.0005e-04, 1.9082e-03,
        1.6664e-02, 3.4231e-02, 0.0000e+00, 4.5794e-03, 7.5270e-03, 6.3216e-03,
        5.1723e-03, 1.8872e-02, 0.0000e+00, 1.7166e-02, 4.6282e-03, 1.4109e-02,
        8.3412e-03, 4.0175e-03, 5.5559e-03, 2.1765e-02, 5.2656e-09, 1.8983e-03,
        1.2783e-02, 3.9692e-03, 1.2224e-02, 6.8619e-03, 3.3184e-03, 6.0384e-03,
        5.0413e-03, 2.1091e-02, 4.2391e-03, 0.0000e+00, 4.0602e-03, 3.4018e-06,
        2.2106e-02, 1.4195e-02, 5.2638e-03, 4.9925e-03, 1.4434e-02, 2.7056e-02,
        5.3126e-03, 1.1585e-02, 5.2733e-03, 3.3394e-05, 7.0872e-03, 1.0555e-02,
        0.0000e+00, 1.5240e-02, 1.8840e-02, 2.4626e-03, 7.4235e-03, 1.1533e-02,
        7.3166e-03, 3.2700e-04, 1.4584e-02, 4.1280e-03, 5.1657e-03, 1.0846e-02,
        5.8089e-03, 7.2987e-03, 9.2555e-03, 4.5452e-03, 1.2172e-02, 9.5547e-03,
        5.6973e-03, 7.2100e-0

 86%|████████▋ | 44/51 [00:06<00:00,  7.32it/s]

KeyboardInterrupt

